# "THE PRICE IS RIGHT" Capstone Project

This week - build a model that predicts how much something costs from a description, based on a scrape of Amazon data


A model that can estimate how much something costs, from its description.

# Order of play

DAY 1: Data Curation  
DAY 2: Data Pre-processing  
DAY 3: Evaluation, Baselines, Traditional ML  
DAY 4: Deep Learning and LLMs  
DAY 5: Fine-tuning a Frontier Model  

## DAY 4: Neural Networks and LLMs

Today we'll work from Traditional ML to Neural Networks to Large Language Models!!

In [1]:
# imports

import os
from dotenv import load_dotenv
from huggingface_hub import login
from pricer.evaluator import evaluate
from litellm import completion
from pricer.items import Item
import numpy as np
from tqdm.notebook import tqdm
import csv
from sklearn.feature_extraction.text import HashingVectorizer
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import CosineAnnealingLR


In [2]:
LITE_MODE = True

load_dotenv(override=True)
hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
username = "ed-donner"
dataset = f"{username}/items_lite" if LITE_MODE else f"{username}/items_full"

train, val, test = Item.from_hub(dataset)

print(f"Loaded {len(train):,} training items, {len(val):,} validation items, {len(test):,} test items")

Loaded 20,000 training items, 1,000 validation items, 1,000 test items


# Before we look at the Artificial Neural Networks

## There is a different kind of Neural Network we could consider

In [4]:
# Write the test set to a CSV

with open('human_in.csv', 'w', encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    for t in test[:100]:
        writer.writerow([t.summary, 0])

In [5]:
# Read it back in

human_predictions = []
with open('human_out.csv', 'r', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        human_predictions.append(float(row[1]))

In [6]:
def human_pricer(item):
    idx = test.index(item)
    return human_predictions[idx]

In [7]:
human = human_pricer(test[0])
actual = test[0].price
print(f"Human predicted {human} for an item that actually costs {actual}")


Human predicted 120.0 for an item that actually costs 219.0


In [8]:
evaluate(human_pricer, test, size=100)

  0%|          | 0/100 [00:00<?, ?it/s]

$99 $184 $12 $15 $18 $10 $119 $135 $6 $270 $643 $329 $15 $26 $24 $18 $29 $25 $25 $53 $35 $126 $25 $127 $273 $398 $55 $6 $101 $51 $30 $5 $35 $9 $10 $419 $25 $11 $186 $33 $161 $51 $23 $155 $150 $4 $31 $18 $115 $82 $25 $111 $410 $75 $67 $34 $8 $10 $122 $28 $116 $17 $19 $60 $599 $60 $160 $355 $75 $34 $17 $2 $70 $76 $41 $9 $226 $5 $5 $4 $0 $7 $5 $74 $7 $10 $68 $74 $5 $3 $17 $45 $5 $16 $0 $153 $2 $122 $150 $355 

# And now - a vanilla Neural Network

During the remainder of this course we will get deeper into how Neural Networks work, and how to train a neural network.

This is just a sneak preview - let's make our own Neural Network, from scratch, using Pytorch.

Use this to get intuition; it's not important to know all about Neural networks at this point..

In [9]:
# Prepare our documents and prices

y = np.array([float(item.price) for item in train])
documents = [item.summary for item in train]

In [10]:
# Use the HashingVectorizer for a Bag of Words model
# Using binary=True with the CountVectorizer makes "one-hot vectors"

np.random.seed(42)
vectorizer = HashingVectorizer(n_features=5000, stop_words='english', binary=True)
X = vectorizer.fit_transform(documents)

In [11]:
# Define the neural network - here is Pytorch code to create a 8 layer neural network

class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_size, 128)
        self.layer2 = nn.Linear(128, 64)
        self.layer3 = nn.Linear(64, 64)
        self.layer4 = nn.Linear(64, 64)
        self.layer5 = nn.Linear(64, 64)
        self.layer6 = nn.Linear(64, 64)
        self.layer7 = nn.Linear(64, 64)
        self.layer8 = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        output1 = self.relu(self.layer1(x))
        output2 = self.relu(self.layer2(output1))
        output3 = self.relu(self.layer3(output2))
        output4 = self.relu(self.layer4(output3))
        output5 = self.relu(self.layer5(output4))
        output6 = self.relu(self.layer6(output5))
        output7 = self.relu(self.layer7(output6))
        output8 = self.layer8(output7)
        return output8

In [12]:
# Convert data to PyTorch tensors
X_train_tensor = torch.FloatTensor(X.toarray())
y_train_tensor = torch.FloatTensor(y).unsqueeze(1)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_tensor, y_train_tensor, test_size=0.01, random_state=42)

# Create the loader
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Initialize the model
input_size = X_train_tensor.shape[1]
model = NeuralNetwork(input_size)

In [13]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Number of trainable parameters: {trainable_params:,}")

Number of trainable parameters: 669,249


In [14]:
# Define loss function and optimizer

loss_function = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# We will do 2 complete runs through the data

EPOCHS = 2

for epoch in range(EPOCHS):
    model.train()
    for batch_X, batch_y in tqdm(train_loader):
        optimizer.zero_grad()

        # The next 4 lines are the 4 stages of training: forward pass, loss calculation, backward pass, optimize
        
        outputs = model(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val)
        val_loss = loss_function(val_outputs, y_val)

    print(f'Epoch [{epoch+1}/{EPOCHS}], Train Loss: {loss.item():.3f}, Val Loss: {val_loss.item():.3f}')

  0%|          | 0/310 [00:00<?, ?it/s]

Epoch [1/2], Train Loss: 11874.317, Val Loss: 20187.166


  0%|          | 0/310 [00:00<?, ?it/s]

Epoch [2/2], Train Loss: 44816.973, Val Loss: 17772.367


In [15]:
def neural_network(item):
    model.eval()
    with torch.no_grad():
        vector = vectorizer.transform([item.summary])
        vector = torch.FloatTensor(vector.toarray())
        result = model(vector)[0].item()
    return max(0, result)

In [16]:
evaluate(neural_network, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$25 $92 $15 $26 $59 $160 $15 $73 $34 $23 $421 $83 $110 $190 $51 $31 $14 $25 $61 $49 $31 $29 $76 $77 $265 $249 $225 $39 $39 $39 $43 $166 $21 $19 $122 $271 $59 $122 $102 $71 $157 $63 $20 $74 $100 $50 $70 $61 $15 $7 $11 $38 $96 $16 $121 $72 $41 $141 $34 $40 $61 $2 $35 $27 $439 $138 $2 $257 $17 $230 $8 $34 $116 $104 $10 $62 $105 $46 $32 $58 $63 $155 $29 $42 $16 $73 $44 $151 $157 $145 $26 $73 $34 $25 $38 $82 $59 $32 $107 $210 $19 $66 $2 $13 $20 $48 $108 $239 $9 $76 $15 $71 $142 $48 $21 $172 $138 $54 $60 $52 $24 $216 $41 $25 $66 $15 $24 $183 $70 $38 $54 $131 $119 $28 $64 $25 $104 $86 $27 $93 $47 $146 $14 $174 $228 $72 $70 $334 $54 $16 $21 $243 $11 $41 $30 $123 $163 $10 $46 $8 $57 $14 $2 $27 $445 $18 $78 $7 $18 $47 $15 $20 $229 $60 $29 $8 $27 $11 $20 $144 $335 $21 $80 $2 $39 $103 $22 $70 $34 $14 $57 $79 $79 $18 $1 $46 $87 $9 $2 $18 

# And now - to the frontier!

Let's see how Frontier models do out of the box; no training, just inference based on their world knowledge.

Tomorrow we will do some training.

In [17]:
def messages_for(item):
    message = f"Estimate the price of this product. Respond with the price, no explanation\n\n{item.summary}"
    return [{"role": "user", "content": message}]

In [18]:
print(test[0].summary)

Title: Excess V2 Distortion/Modulation Pedal  
Category: Music Pedals  
Brand: Old Blood Noise  
Description: A versatile pedal offering distortion and three modulation modes—delay, chorus, and harmonized fifths—with full control over signal routing and expression.  
Details: Features include separate gain, tone, and volume controls; time, depth, and volume per modulation; order switching, soft‑touch bypass, and expression jack for dynamic control.


In [19]:
messages_for(test[0])

[{'role': 'user',
  'content': 'Estimate the price of this product. Respond with the price, no explanation\n\nTitle: Excess V2 Distortion/Modulation Pedal  \nCategory: Music Pedals  \nBrand: Old Blood Noise  \nDescription: A versatile pedal offering distortion and three modulation modes—delay, chorus, and harmonized fifths—with full control over signal routing and expression.  \nDetails: Features include separate gain, tone, and volume controls; time, depth, and volume per modulation; order switching, soft‑touch bypass, and expression jack for dynamic control.'}]

In [20]:
# The function for gpt-4.1-nano

def gpt_4__1_nano(item):
    response = completion(model="openai/gpt-4.1-nano", messages=messages_for(item))
    return response.choices[0].message.content

In [21]:
gpt_4__1_nano(test[0])

'$250'

In [22]:
test[0].price

219.0

In [23]:
evaluate(gpt_4__1_nano, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$19 $34 $25 $80 $120 $80 $6 $65 $9 $2170 $263 $29 $30 $4 $19 $13 $71 $5 $140 $1 $59 $26 $35 $25 $82 $304 $405 $5 $151 $64 $30 $10 $10 $60 $35 $119 $90 $31 $36 $23 $175 $45 $20 $105 $70 $0 $27 $13 $65 $52 $23 $105 $225 $20 $97 $16 $8 $40 $52 $3 $116 $23 $61 $40 $179 $10 $90 $265 $25 $74 $17 $8 $30 $6 $15 $21 $76 $5 $3 $3 $30 $8 $15 $74 $12 $0 $32 $44 $40 $11 $3 $25 $5 $19 $2 $108 $6 $7 $20 $425 $50 $33 $12 $11 $1 $82 $10 $370 $19 $49 $10 $236 $119 $48 $54 $180 $15 $5 $94 $47 $29 $411 $50 $16 $0 $10 $5 $151 $29 $54 $79 $13 $65 $0 $85 $0 $55 $0 $58 $62 $86 $150 $70 $4 $134 $118 $20 $60 $115 $13 $6 $144 $2 $4860 $1 $29 $101 $41 $30 $5 $211 $17 $6 $2 $340 $7 $752 $30 $15 $5 $5 $3 $20 $8 $27 $201 $3 $57 $34 $33 $546 $20 $150 $99 $20 $18 $73 $8 $0 $2 $5 $49 $25 $11 $21 $120 $10 $70 $16 $1 

In [ ]:
#def claude_opus_4_5(item):
#    response = completion(model="anthropic/claude-opus-4-5", messages=messages_for(item))
#    return response.choices[0].message.content

In [ ]:
#evaluate(claude_opus_4_5, test)

In [ ]:
#def gemini_3_pro_preview(item):
#    response = completion(model="gemini/gemini-3-pro-preview", messages=messages_for(item), reasoning_effort='low')
#    return response.choices[0].message.content

In [ ]:
#evaluate(gemini_3_pro_preview, test, size=50, workers=2)

In [24]:
def gemini_2__5_flash_lite(item):
    response = completion(model="gemini/gemini-2.5-flash-lite", messages=messages_for(item))
    return response.choices[0].message.content

In [ ]:
#evaluate(gemini_2__5_flash_lite, test)

In [ ]:

#def grok_4__1_fast(item):
#    response = completion(model="xai/grok-4-1-fast-non-reasoning", messages=messages_for(item), seed=42)
#    return response.choices[0].message.content

In [ ]:
#evaluate(grok_4__1_fast, test)

In [27]:
# The function for gpt-5.1

def gpt_5__1(item):
    response = completion(model="gpt-5.1", messages=messages_for(item), reasoning_effort='high', seed=42)
    return response.choices[0].message.content


In [28]:
evaluate(gpt_5__1, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$10 $74 $5 $0 $10 $140 $64 $95 $11 $69 $87 $20 $0 $9 $49 $8 $11 $20 $30 $69 $6 $16 $20 $5 $53 $203 $165 $1 $71 $63 $13 $30 $29 $40 $5 $69 $50 $36 $44 $18 $160 $45 $15 $105 $40 $0 $7 $2 $65 $28 $26 $107 $146 $0 $17 $29 $3 $90 $7 $2 $106 $48 $37 $30 $21 $0 $50 $305 $5 $44 $17 $3 $60 $4 $25 $17 $26 $2 $2 $7 $0 $2 $10 $74 $14 $25 $58 $66 $0 $16 $3 $25 $5 $0 $0 $99 $1 $87 $110 $255 $0 $13 $7 $19 $49 $142 $15 $340 $6 $119 $10 $66 $9 $58 $54 $0 $10 $0 $14 $547 $7 $61 $10 $46 $10 $15 $2 $21 $11 $49 $104 $2 $5 $0 $45 $2 $15 $10 $71 $2 $6 $149 $35 $5 $34 $8 $5 $110 $15 $8 $6 $93 $27 $60 $4 $21 $41 $36 $55 $0 $110 $18 $8 $0 $59 $2 $552 $20 $0 $3 $10 $3 $220 $13 $62 $9 $5 $37 $4 $3 $134 $15 $240 $49 $22 $3 $63 $17 $10 $7 $5 $4 $10 $31 $54 $50 $21 $20 $16 $11 